In [5]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Optional
from dotenv import load_dotenv
from boeing_chat_model import BoeingChatModel
import os
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [2]:
load_dotenv()

PAT = os.getenv("PAT")

In [3]:
model = BoeingChatModel(
    udal_pat=PAT
)

In [4]:
class BlogState(BaseModel):

    topic: str = Field(..., description="The topic of the blog")
    outline: Optional[str] = Field(None, description="The outline of the blog")
    blog: Optional[str] = Field(None, description="The blog content")

In [6]:
def get_outline(state: BlogState) -> BlogState:
    topic = state.topic
    parser = StrOutputParser()
    prompt = PromptTemplate(
        template="Generate an outline for a Blog on the topic: {topic}",
        input_variables=['topic']
    )
    chain = prompt | model | parser
    outline = chain.invoke({'topic': topic})
    state.outline = outline
    return state

In [10]:
def get_blog(state: BlogState) -> BlogState:
    topic = state.topic
    outline = state.outline

    parser = StrOutputParser()
    prompt = PromptTemplate(
        template="""
        Generate a Blog on the topic:
        {topic}
        With this outline:
        {outline}
        """,
        input_variables=['topic', 'outline']
    )
    chain = prompt | model | parser

    blog = chain.invoke({'topic': topic, 'outline': outline})
    state.blog = blog
    return state

In [11]:
graph = StateGraph(BlogState)

graph.add_node("get_outline", get_outline)
graph.add_node("get_blog", get_blog)

graph.add_edge(START, "get_outline")
graph.add_edge("get_outline", "get_blog")
graph.add_edge("get_blog", END)

workflow = graph.compile()

In [12]:
initial_state = {"topic": "Cricket"}
final_state = workflow.invoke(initial_state)
final_state

{'topic': 'Cricket',
 'outline': '# Blog Outline: Cricket\n\n## 1. Working Title Ideas\n- The Ultimate Guide to Cricket: Rules, Formats, and Iconic Moments\n- Cricket for Everyone: From Backyard Tosses to World Cups\n- How Cricket Works: A Beginner’s Guide and Advanced Insights\n\n---\n\n## 2. Target Audience\n- Complete beginners wanting to understand the game\n- Casual fans seeking deeper context (formats, strategies, major events)\n- Enthusiasts interested in player profiles, stats, and history\n- Coaches and club players looking for practical tips\n\n---\n\n## 3. Suggested Structure & Section Breakdown\n\n### I. Introduction (150–250 words)\n- Brief hook: why cricket matters globally\n- What the reader will learn\n- Quick overview of the main formats: Test, ODI, T20\n\n### II. What Is Cricket? (300–400 words)\n- Basic objective and structure of a game\n- Roles: batter, bowler, fielder, wicket-keeper\n- Playing conditions: pitch, overs, innings\n\n### III. Formats Explained (400–600

In [16]:
print(final_state['blog'])

# The Ultimate Guide to Cricket: Rules, Formats, and How to Get Started

Suggested meta title: Cricket Explained: Rules, Formats & How to Get Started  
Suggested meta description: Learn cricket basics, formats (Test/ODI/T20), rules, tactics, top players, and how to start playing. A complete guide for fans and beginners.

---

I. Introduction (about 180 words)
---------------------------------

Cricket is more than a sport — it’s a global pastime that blends strategy, athletic skill, and drama. From sun-drenched village greens to packed international stadiums, cricket has evolved into formats that suit both marathon contests and short, explosive entertainment. Whether you’ve walked past a backyard game, caught a T20 highlight on social media, or tuned in to a historic Test match, cricket has a way of drawing people in with its unique rhythm: patient build-ups, sudden turning points, and moments of individual brilliance.

In this guide you’ll learn what cricket is, how different formats 